In [1]:
@show versioninfo()
@show pwd()
@show Threads.nthreads()
import Pkg
Pkg.activate("..")

Julia Version 1.6.3
Commit ae8452a9e0 (2021-09-23 17:34 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.5.0)
  CPU: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, icelake-client)
Environment:
  JULIA_NUM_THREADS = 
versioninfo() = nothing
pwd() = "/Users/jieqiushao/Code/PRONTO.jl/scripts"
Threads.nthreads() = 

1


  Activating environment at `~/Code/PRONTO.jl/Project.toml`


In [2]:
Pkg.update()

    Updating registry at `~/.julia/registries/General`


  No Changes to `~/Code/PRONTO.jl/Project.toml`
  No Changes to `~/Code/PRONTO.jl/Manifest.toml`


In [3]:
using DifferentialEquations
using LinearAlgebra
# using Plots
using Interpolations

H0 = [0 0 1 0;
      0 0 0 -1;
     -1 0 0 0;
      0 1 0 0]

H1 = [0 -1 0 0;
      1 0 0 0;
      0 0 0 -1;
      0 0 1 0]

 f(x,u,t) =(H0 + u*H1)*x;

fx(x,u,t) = H0 + u*H1;
fu(x,u,t) = H1*x;

fxx = reshape(zeros(8,8), (4,4,4))
fxu = reshape(H1', (4,1,4))
fuu = reshape(zeros(4,1), (1,1,4))

T = 15
tspan = (0.0,T)

dmu(mu,_,t) = 0*t

mu = solve(ODEProblem(dmu,0,tspan))

dal(al,_,t) = f(al,mu(t))

al0 = [1;0;0;0]

al = solve(ODEProblem(dal,al0,tspan))

function inprod(x)
    a = x[1:2]
    b = x[3:4]
    P = [a*a'+b*b' -(a*b'+b*a');
         a*b'+b*a' a*a'+b*b']
    return P
end

Pr_terminal(al) = I(4) - inprod(al)

Ar(al,mu,t) = fx(al,mu,t)
Br(al,mu,t) = fu(al,mu,t)
Qr(al,mu,t) = I(4) - inprod(al)
Rr(al,mu,t) = 1
    
Ar(t) = Ar(al(t),mu(t),t)
Br(t) = Br(al(t),mu(t),t)
Qr(t) = Qr(al(t),mu(t),t)

Kr(Pr,Rr,Br,t)= Rr\(Br'*Pr)
    
Kr(t) = Kr(Pr(t),Rr(t),Br(t),t)

dPr(Pr,_,t) = Ar(T-t)'*Pr + Pr*Ar(T-t) - Kr(Pr,Rr(T-t),Br(T-t),T-t)'*Rr(T-t)*Kr(Pr,Rr(T-t),Br(T-t),T-t) + Qr(T-t)
Pr(t) = Pr_back(T-t)
    
u(x,Kr,al,mu,t) = mu - Kr*(x - al)

u(t) = u(x(t),Kr(t),al(t),mu(t),t)
    
dx(x,_,t) = f(x,u(x,Kr(Pr(t),Rr(t),Br(t),t),al(t),mu(t),t))

x0 = [0;1;0;0]

dL(_,_,t) = 0.01/2*u(t)^2
    
P_terminal = I(4) - inprod([1 0 0 0])

A(x,u,t) = fx(x,u,t)
B(x,u,t) = fu(x,u,t)
a(x,u,t) = [0;0;0;0]
b(x,u,t) = 0.01*u
Q(t) = zeros(4,4)
S(t) = zeros(4,1)
R(t) = 0.01

A(t) = A(x(t),u(t),t)
B(t) = B(x(t),u(t),t)
a(t) = a(x(t),u(t),t)
b(t) = b(x(t),u(t),t)

dq(q,_,t) = (A(T-t) - B(T-t)*Kr(T-t))'*q + a(T-t) - Kr(T-t)'*b(T-t)

q(t) = q_back(T-t)

Qo(t) = Q(t) + q(t)[1]*fxx[:,:,1] + q(t)[2]*fxx[:,:,2] + q(t)[3]*fxx[:,:,3] + q(t)[4]*fxx[:,:,4]
So(t) = S(t) + q(t)[1]*fxu[:,:,1] + q(t)[2]*fxu[:,:,2] + q(t)[3]*fxu[:,:,3] + q(t)[4]*fxu[:,:,4]
Ro(t) = R(t) + q(t)[1]*fuu[:,:,1][1,1] + q(t)[2]*fuu[:,:,2][1,1] + q(t)[3]*fuu[:,:,3][1,1] + q(t)[4]*fuu[:,:,4][1,1]

# GD
Ko(P,R,S,B,t)= R\(S' + B'*P)

Ko(t) = Ko(P(t),R(t),S(t),B(t),t)

dP(P,_,t) = A(T-t)'*P + P*A(T-t) - Ko(P,R(T-t),S(T-t),B(T-t),T-t)'*R(T-t)*Ko(P,R(T-t),S(T-t),B(T-t),T-t) + Q(T-t)
P(t)=P_back(T-t)

PT = [0 0 0 0;
      0 1 0 0;
      0 0 0 0;
      0 0 0 1]

dr(r,_,t) = (A(T-t) - B(T-t)*Ko(T-t))'*r + a(T-t) - vec(Ko(T-t)'*b(T-t))
r(t)=r_back(T-t)

vo(R,B,r,b,t) = -R\(B'*r + b)

vo(t) = vo(R(t),B(t),r(t),b(t),t)

v(z,Ko,vo,t) = (-Ko*z)[1] + vo

v(t) = v(z(t),Ko(t),vo(t),t)

dz(z,_,t) = A(t)*z + B(t)*v(z,Ko(t),vo(t),t)

z0 = 0*x0

dy1(_,_,t) = a(t)'*z(t) + b(t)'*v(t)

dy2(_,_,t) = z(t)'*Qo(t)*z(t) + (2*z(t)'*So(t)*v(t))'[1] + v(t)'*Ro(t)*v(t)

gamma = 1

al1(x,z,t)    = x + gamma*z
mu1(u,v,t)    = u + gamma*v
u1(x1,Kr,al1,mu1,t) = mu1 - Kr*(x1 - al1)

al1(t) = al1(x(t),z(t),t)
mu1(t) = mu1(u(t),v(t),t) 
u1(t) = u1(x1(t),Kr(t),al1(t),mu1(t),t);


dx1(x1,_,t) = f(x1,u1(x1,Kr(t),al1(x(t),z(t),t),mu1(u(t),v(t),t)))
dL1(_,_,t) = 0.01/2*u1(t)^2

# Newt
Ko2(P2,Ro,So,B,t)= Ro\(So' + B'*P2)

Ko2(t) = Ko2(P2(t),Ro(t),So(t),B(t),t)

dP2(P2,_,t) = A(T-t)'*P2 + P2*A(T-t) - Ko2(P2,Ro(T-t),So(T-t),B(T-t),T-t)'*Ro(T-t)*Ko2(P2,Ro(T-t),So(T-t),B(T-t)) + Qo(T-t)
P2(t)=P2_back(T-t)

P2T = [0 0 0 0;
      0 1 0 0;
      0 0 0 0;
      0 0 0 1]

dr2(r2,_,t) = (A(T-t) - B(T-t)*Ko2(T-t))'*r2 + a(T-t) - vec(Ko2(T-t)'*b(T-t))
r2(t)=r2_back(T-t)

vo2(Ro,B,r2,b,t) = -Ro\(B'*r2 + b)

vo2(t) = vo2(Ro(t),B(t),r2(t),b(t),t)

v2(z2,Ko2,vo2,t) = (-Ko2*z2)[1] + vo2

v2(t) = v2(z2(t),Ko2(t),vo2(t),t)

dz2(z2,_,t) = A(t)*z2 + B(t)*v2(z2,Ko2(t),vo2(t))

z20 = 0*x0

dy12(_,_,t) = a(t)'*z2(t) + b(t)'*v2(t)

dy22(_,_,t) = z2(t)'*Qo(t)*z2(t) + (2*z2(t)'*So(t)*v2(t))'[1] + v2(t)'*Ro(t)*v2(t)

gamma = 1

al2(x,z2,t)    = x + gamma*z2
mu2(u,v2,t)    = u + gamma*v2
u2(x2,Kr,al2,mu2,t) = mu2 - Kr*(x2 - al2)

al2(t) = al2(x(t),z2(t),t)
mu2(t) = mu2(u(t),v2(t),t)
u2(t) = u2(x2(t),Kr(t),al2(t),mu2(t),t);


dx2(x2,_,t) = f(x2,u2(x2,Kr(t),al2(x(t),z2(t),t),mu2(u(t),v2(t),t)))
dL2(_,_,t) = 0.01/2*u2(t)^2

MethodError: MethodError: no method matching f(::Vector{Float64}, ::Float64)
Closest candidates are:
  f(::Any, ::Any, !Matched::Any) at /Users/jieqiushao/Code/PRONTO.jl/scripts/2spin_PRONTO_v0.ipynb:16

In [4]:
global Pr_back
global x
global L
global h
global q_back
global P_back
global P2_back
global r_back
global r2_back
global z
global z2
global y1
global y12
global y2
global y22
global Dh
global Dg2
global x1
global x2
global L1
global L2
global g
global tt
global utt
CostH = zeros(10)
CostG = zeros(10)
Descent = zeros(10)
Gam = zeros(10)
Method = zeros(10)

tt = LinRange(0, T, 1000)
utt = zeros(1000)

for iter = 1:10

    Pr_back = solve(ODEProblem(dPr,Pr_terminal(al(T)),tspan))

    x = solve(ODEProblem(dx,x0,tspan))

    L = solve(ODEProblem(dL,0,tspan))

    h = L(T) + 1/2*x(T)'*P_terminal*x(T)
    
    CostH[iter] = h

    q_back = solve(ODEProblem(dq,P_terminal*al(T),tspan))
    
#     try
        
    P2_back = solve(ODEProblem(dP2,P2T,tspan),maxiters = 1e3)
    
    if abs(P2_back.destats.maxeig) < 1e8
        
        r2_back = solve(ODEProblem(dr2,P_terminal*al(T),tspan))
        
        z2 = solve(ODEProblem(dz2,z20,tspan))
        
        y12 = solve(ODEProblem(dy12,0,tspan))
        
        y22 = solve(ODEProblem(dy22,0,tspan))
        
        Dh = y12(T) + (P_terminal*al(T))'*z2(T)
        Dg2 = y22(T) + z2(T)'*P_terminal*z2(T)
    
        Descent[iter] = -Dh
        
        if Descent[iter] < 1e-5
            break
        end
        
        Method[iter] = 2

        x2 = al
        L2 = L
        g = h
        gamma = 1

        for j = 1:5

            x2 = solve(ODEProblem(dx2,x0,tspan))

            L2 = solve(ODEProblem(dL2,0,tspan))

            g = L2(T) + 1/2*x2(T)'*P_terminal*x2(T)
    
            if h-g >= -0.4*gamma*Dh
                Gam[iter] = gamma
                break
            else
                gamma = 0.7*gamma
            end
    
        end
    
        CostG[iter] = g

    
        for counter = 1:1000
            utt[counter] = u2(tt[counter])
        end
    
        mu = LinearInterpolation(tt,utt)
        al = x2
        
#     catch 
    else 
        
        P_back = solve(ODEProblem(dP,PT,tspan))

        r_back = solve(ODEProblem(dr,P_terminal*al(T),tspan))

        z = solve(ODEProblem(dz,z0,tspan))

        y1 = solve(ODEProblem(dy1,0,tspan))

        y2 = solve(ODEProblem(dy2,0,tspan))

        Dh = y1(T) + (P_terminal*al(T))'*z(T)
        Dg2 = y2(T) + z(T)'*P_terminal*z(T)
    
        Descent[iter] = -Dh
        
        if Descent[iter] < 1e-5
            break
        end
        
        Method[iter] = 1

        x1 = al
        L1 = L
        g = h
        gamma = 1

        for j = 1:5

            x1 = solve(ODEProblem(dx1,x0,tspan))

            L1 = solve(ODEProblem(dL1,0,tspan))

            g = L1(T) + 1/2*x1(T)'*P_terminal*x1(T)
    
            if h-g >= -0.4*gamma*Dh
                Gam[iter] = gamma
                break
            else
                gamma = 0.7*gamma
            end
    
        end
    
        CostG[iter] = g

    
        for counter = 1:1000
            utt[counter] = u1(tt[counter])
        end
    
        mu = LinearInterpolation(tt,utt)
        al = x1
        
    end
    
end

UndefVarError: UndefVarError: al not defined

In [5]:
Descent

10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [6]:
CostG

10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [7]:
Method

10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [8]:
Gam

10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [9]:
plot(al)

UndefVarError: UndefVarError: plot not defined

In [10]:
plot(tt,utt)

UndefVarError: UndefVarError: plot not defined